In [48]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler

In [55]:
train = pd.read_csv("train.csv")

In [56]:
train.head(20)

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.0,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.0,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.0,-7.349849,0,0_0_4
5,5,0,0,0.00,0,1.000635,13552875.92,NaN,NaN,0.999779,1962.72,1.000635,5647.65,1.0,6.779432,0,0_0_5
6,6,0,0,969969.40,1,1.000115,3647503.98,NaN,NaN,0.999506,6663.16,1.000283,3810.48,1.0,-2.499819,0,0_0_6
7,7,0,0,9412959.10,1,0.999818,21261245.87,NaN,NaN,0.999741,5139.20,1.000130,2570.60,1.0,-1.959801,0,0_0_7
8,8,0,0,2394875.85,1,0.999916,9473209.08,NaN,NaN,0.999022,52011.60,1.000041,2169.36,1.0,-5.970001,0,0_0_8
9,9,0,0,3039700.65,-1,1.000969,6248958.45,NaN,NaN,0.999354,6191.00,1.000646,6199.00,1.0,7.970333,0,0_0_9


In [57]:
train.tail(20)

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
5237960,180,480,540,563043.09,-1,0.998141,6.717567e+06,0.996492,0.996767,0.997866,29322.32,0.998141,13757.70,0.998054,15.909672,26454,480_540_180
5237961,181,480,540,498430.75,1,1.001248,1.384217e+07,1.001527,1.001304,1.001081,1256.85,1.001304,59803.47,1.001085,-2.139807,26454,480_540_181
5237962,182,480,540,2685032.08,1,0.999491,3.848839e+07,1.000210,1.000210,0.999311,210094.58,0.999491,134544.78,0.999421,3.750324,26454,480_540_182
5237963,183,480,540,571821.88,-1,0.996694,9.752797e+07,0.996468,0.996468,0.996694,19896.75,0.996807,83487.36,0.996716,1.029968,26454,480_540_183
5237964,184,480,540,283509.38,1,0.996650,1.590741e+07,0.998372,0.997156,0.996447,63619.51,0.996650,75336.10,0.996540,-4.770160,26454,480_540_184
5237965,185,480,540,3018493.53,-1,1.000057,2.723894e+07,0.997937,0.998915,1.000029,29442.40,1.000301,225663.69,1.000061,-7.209778,26454,480_540_185
5237966,186,480,540,2106025.88,-1,0.999780,1.988346e+07,0.998713,0.998713,0.999780,174758.40,0.999933,516941.19,0.999819,-9.809732,26454,480_540_186
5237967,187,480,540,3791745.66,1,0.999641,3.380628e+07,1.000641,1.000564,0.999564,24031.50,0.999641,73788.88,0.999583,0.289679,26454,480_540_187
5237968,188,480,540,18930.38,-1,0.999741,6.465049e+06,0.999741,0.999741,0.999741,117186.86,1.000297,217260.00,0.999936,-0.699758,26454,480_540_188
5237969,189,480,540,956757.96,1,1.000715,3.710938e+07,1.001167,1.001054,1.000602,788361.12,1.000715,49599.20,1.000708,6.959438,26454,480_540_189


In [58]:
num_rows, num_columns = train.shape
print("Number of Rows: ", num_rows)
print("Number of Columns: ", num_columns)

Number of Rows:  5237980
Number of Columns:  17


In [59]:
train = train.dropna(subset=["target"])

# 🔁 Reset the index of the DataFrame and apply the changes in place
train.reset_index(drop=True, inplace=True)

num_rows, num_columns = train.shape
print("Number of Rows: ", num_rows)
print("Number of Columns: ", num_columns)

Number of Rows:  5237892
Number of Columns:  17


In [60]:
pos_changes = train['reference_price'].diff().clip(lower=1)
neg_changes = train['reference_price'].diff().clip(upper=1)

train['RSI'] = 1 - (100/(1 + pos_changes/neg_changes))

In [61]:
#train['date_id'] = pd.to_datetime(train['date_id'])

In [62]:
# Define the window size for the moving average
window_size = 2
# Compute the moving averages for each stock and the 'wap' column
train['SMA'] = train.groupby('stock_id')['wap'].transform(lambda x: x.rolling(window=window_size,  min_periods=1).mean())

In [63]:
daily_wap_sum = train.groupby('date_id')['wap'].sum()
rows_per_day = train.groupby('date_id').size()

overall_average_wap = daily_wap_sum / rows_per_day

train = train.merge(overall_average_wap.reset_index(name='daily_average_wap'), on='date_id', how='left')

In [64]:
train.head(20)

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id,RSI,SMA,daily_average_wap
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0,0_0_0,NaN,1.0,1.000021
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,0,0_0_1,0.991601,1.0,1.000021
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.0,-8.389950,0,0_0_2,1.033511,1.0,1.000021
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.0,-4.010200,0,0_0_3,0.939037,1.0,1.000021
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.0,-7.349849,0,0_0_4,1.063941,1.0,1.000021
5,5,0,0,0.00,0,1.000635,13552875.92,NaN,NaN,0.999779,1962.72,1.000635,5647.65,1.0,6.779432,0,0_0_5,0.889822,1.0,1.000021
6,6,0,0,969969.40,1,1.000115,3647503.98,NaN,NaN,0.999506,6663.16,1.000283,3810.48,1.0,-2.499819,0,0_0_6,1.052027,1.0,1.000021
7,7,0,0,9412959.10,1,0.999818,21261245.87,NaN,NaN,0.999741,5139.20,1.000130,2570.60,1.0,-1.959801,0,0_0_7,1.029709,1.0,1.000021
8,8,0,0,2394875.85,1,0.999916,9473209.08,NaN,NaN,0.999022,52011.60,1.000041,2169.36,1.0,-5.970001,0,0_0_8,0.990201,1.0,1.000021
9,9,0,0,3039700.65,-1,1.000969,6248958.45,NaN,NaN,0.999354,6191.00,1.000646,6199.00,1.0,7.970333,0,0_0_9,0.894811,1.0,1.000021


In [47]:
train.tail(20)

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id,RSI,SMA,daily_average_wap
5237842,150,480,540,3871102.65,-1,0.997898,1.714805e+08,0.997898,0.997898,0.997860,26060.02,0.997976,29159.65,0.997914,-4.720092,26454,480_540_150,1.370266,0.997898,0.999562
5237843,151,480,540,2291517.33,1,1.001291,1.221302e+08,1.001680,1.001680,1.001097,727720.00,1.001291,729355.62,1.001194,-1.659989,26454,480_540_151,0.661847,1.001194,0.999562
5237844,152,480,540,3723029.37,-1,1.000466,4.895921e+07,0.999471,0.999471,1.000342,245762.42,1.000508,32323.48,1.000488,-8.919835,26454,480_540_152,1.082568,1.000491,0.999562
5237845,153,480,540,0.00,0,1.001467,8.302759e+06,1.001467,1.001467,1.001467,63179.82,1.002001,52640.34,1.001758,-6.979704,26454,480_540_153,0.900000,1.002327,0.999562
5237846,154,480,540,0.00,0,1.000702,1.791880e+07,1.000702,1.000702,1.000326,171543.40,1.000702,356547.39,1.000448,-3.730059,26454,480_540_154,1.076559,1.000450,0.999562
5237847,155,480,540,452097.69,1,0.998887,7.218048e+06,0.999366,0.999366,0.998767,54401.43,0.998887,3499.44,0.998880,0.120401,26454,480_540_155,1.181830,0.998876,0.999562
5237848,156,480,540,424588.88,1,0.995629,9.863525e+06,0.998316,0.995629,0.994286,272001.80,0.995629,634451.61,0.994689,10.700226,26454,480_540_156,1.326865,0.994890,0.999562
5237849,157,480,540,5413573.11,1,1.002186,4.476286e+07,1.003169,1.003169,1.002038,25680.06,1.002186,5299.84,1.002160,1.989603,26454,480_540_157,0.348571,1.002156,0.999562
5237850,158,480,540,0.00,0,0.999648,8.578806e+06,0.999648,0.999648,0.999648,47588.31,0.999874,38768.09,0.999773,-2.260208,26454,480_540_158,1.254446,0.999896,0.999562
5237851,159,480,540,291140.65,-1,1.001038,2.267847e+07,1.000372,1.000752,1.001038,85284.90,1.001228,161229.61,1.001103,-3.209710,26454,480_540_159,0.861193,1.001182,0.999562


In [66]:
#Doesn't seem like normalization is used, this here just for record keeping for the time
numerical_columns = ['date_id', 'seconds_in_bucket', 'imbalance_size', 'imbalance_buy_sell_flag',
                     'reference_price', 'matched_size', 'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price', 
                     'ask_size', 'wap', 'time_id']

scaler = MinMaxScaler()
normalized_train = train
normalized_train[numerical_columns] = scaler.fit_transform(normalized_train[numerical_columns])

In [67]:
print(normalized_train)

         stock_id  date_id  seconds_in_bucket  imbalance_size  \
0               0      0.0                0.0        0.001067   
1               1      0.0                0.0        0.000056   
2               2      0.0                0.0        0.000102   
3               3      0.0                0.0        0.003997   
4               4      0.0                0.0        0.000150   
...           ...      ...                ...             ...   
5237887       195      1.0                1.0        0.000818   
5237888       196      1.0                1.0        0.000117   
5237889       197      1.0                1.0        0.000000   
5237890       198      1.0                1.0        0.000336   
5237891       199      1.0                1.0        0.000632   

         imbalance_buy_sell_flag  reference_price  matched_size  far_price  \
0                            1.0         0.453767      0.001734        NaN   
1                            0.0         0.454358      0.000212

In [ ]:
#potentially remove row id? 
#what to do about the buy/sell flag? (could confuse training with opposite results having similar looking data)
#clearly is regression